In [4]:
using CMBLensing

# Field Basics

## Base Fields

Much of CMBLensing is based on manipilating CMB "fields", like temperature, Q or U polarization, or the lensing potential $\phi$. These types are all encompassed by the abstract type `Field`, with some concrete examples including `FlatMap` for a flat-sky map projection, or `FlatQUMap` for Q/U polarization, etc...

`Flat` fields are just thing wrappers around Julia arrays, e.g.

In [5]:
Ix = rand(2,2)

2×2 Array{Float64,2}:
 0.732042  0.182965
 0.998787  0.732902

In [6]:
f = FlatMap(Ix)

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 0.7320418196232326 
 0.9987870573048432 
 0.18296504638359612
 0.7329019559062799 

You can see the pixels in the 2x2 map have been splayed out into a length-4 array (at least for displaying purposes). This is intentional, as even though the maps themselves are two-dimensional, it is extremely useful conceptually to think of fields as vectors (which they are, in fact, as they form an [abstract vector space](https://en.wikipedia.org/wiki/Vector_space)). This tie to vector spaces is deeply rooted in CMBLensing, to the extent that `Field` objects are a subtype of Julia's own `AbstractVector` type, 

In [7]:
f isa AbstractVector

true

That said, the data itself is stored as the original 2x2 matrix, and can be accessed as follows,

In [8]:
f.Ix

2×2 Array{Float64,2}:
 0.732042  0.182965
 0.998787  0.732902

But since `Fields` are vectors, they can be tranposed,

In [9]:
f'

1×4 LinearAlgebra.Adjoint{Float64,FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}}:
 0.732042  0.998787  0.182965  0.732902

inner products can be computed,

In [10]:
f' * f

1.7803892228763662e-7

and they can be added with each other as well as multiplied by scalars,

In [11]:
2*f+f

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 2.1961254588696977
 2.9963611719145296
 0.5488951391507884
 2.19870586771884  

## Diagonal operators

Vector spaces have linear operators which act on the vectors. Linear operators correpsond to matrices, thus for a map with $N$ total pixels, a general linear operator would be an $N$-by-$N$ matrix, which for even modest map sizes becomes far too large to actually store. Thus, an important class of linear operators are ones which are diagonal, since these can actually be stored. CMBLensing uses Julia's builtin `Diagonal` to represent these. `Diagonal(f)` takes a vector `f` and puts it on the diagonal of the matrix:

In [12]:
Diagonal(f)

4×4 Diagonal{Float64,FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}}:
 0.732042   ⋅         ⋅         ⋅      
  ⋅        0.998787   ⋅         ⋅      
  ⋅         ⋅        0.182965   ⋅      
  ⋅         ⋅         ⋅        0.732902

Multiplying this operator by the original map is then a matrix-vector product:

In [13]:
Diagonal(f) * f

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 0.5358852256772934 
 0.9975755858396681 
 0.03347620819815148
 0.5371452769712507 

Note that this is also equal to the the pointwise multiplication of `f` with itself:

In [14]:
f .* f

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 0.5358852256772934 
 0.9975755858396681 
 0.03347620819815148
 0.5371452769712507 

## Field Tuples

You can put `Fields` together into tuples. For example, 

In [158]:
a = FlatMap(rand(2,2))
b = FlatMap(rand(2,2));

In [159]:
FieldTuple(a,b)

8-element Field2Tuple{CMBLensing.BasisTuple{Tuple{Map,Map}}, Float64}:
 0.5505235821680978  
 0.950054953812133   
 0.014057456492081677
 0.36803238748144085 
 0.006160415105918249
 0.36772974041695905 
 0.7377151845932439  
 0.3910706904663759  

The components can also have names:

In [160]:
ft = FieldTuple(a=a, b=b)

8-element Field2Tuple{(:a, :b), CMBLensing.BasisTuple{Tuple{Map,Map}}, Float64}:
 0.5505235821680978  
 0.950054953812133   
 0.014057456492081677
 0.36803238748144085 
 0.006160415105918249
 0.36772974041695905 
 0.7377151845932439  
 0.3910706904663759  

which can be accessed later:

In [162]:
ft.a

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 0.5505235821680978  
 0.950054953812133   
 0.014057456492081677
 0.36803238748144085 

`FieldTuples` have all of the same behavior of individual fields. Indeed, spin fields like QU or IQU are simply special `FieldTuples`:

In [166]:
FlatQUMap(Q=a, U=b) isa FieldTuple

true

## Field Vectors

*in progress*

## Basis Conversion

All fields are tagged as to which basis they are stored in. You can convert them to other bases by calling the basis type on them:

In [58]:
f

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 0.7320418196232326 
 0.9987870573048432 
 0.18296504638359612
 0.7329019559062799 

In [59]:
f′ = Fourier(f)

4-element FlatFourier{2×2 map, 1′ pixels, fourier∂, Array{Complex{Float64}}}:
   3.564317699947486e-8 + 0.0im
 -1.0998296613404274e-8 + 0.0im
   1.097512962242427e-8 + 0.0im
   3.813755469023178e-9 + 0.0im

Basis conversion is usually done automatically for you. E.g. here `f′` is automatically converted to a `FlatMap` before addition:

In [60]:
f + f′

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 1.4640836392464651 
 1.9975741146096864 
 0.36593009276719235
 1.4658039118125599 

A key feature of `Diagonal` operators is they convert the field they are acting on to the right basis before multiplication:

In [61]:
Diagonal(f) * f′

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 0.5358852256772935
 0.9975755858396682
 0.0334762081981515
 0.5371452769712508

A `FlatMap` times a `FlatFourier` doesn't have a natural linear algebra meaning so its an error:

In [63]:
f * f′

MethodError: MethodError: no method matching *(::FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}, ::FlatFourier{2×2 map, 1′ pixels, fourier∂, Array{Complex{Float64}}})
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:529
  *(!Matched::PyCall.PyObject, ::Any) at /home/marius/.julia/packages/PyCall/ttONZ/src/pyoperators.jl:13
  *(!Matched::FuncOp, ::Field) at /home/marius/work/baylens/src/specialops.jl:105
  ...

## Properties and indices

`FlatMap` and `FlatFourier` can be indexed directly like arrays. If given 1D indices, this is the index into the vector representation:

In [72]:
f

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 0.7320418196232326 
 0.9987870573048432 
 0.18296504638359612
 0.7329019559062799 

In [73]:
f[1], f[2], f[3], f[4]

(0.7320418196232326, 0.9987870573048432, 0.18296504638359612, 0.7329019559062799)

In [74]:
f[5]

BoundsError: BoundsError: attempt to access 2×2 Array{Float64,2} at index [5]

Or with a 2D index, this indexes directly into the 2D map:

In [83]:
f[1,1], f[2,1], f[1,2], f[2,2]

(0.7320418196232326, 0.9987870573048432, 0.18296504638359612, 0.7329019559062799)

*Note:* there is no overhead to indexing `f` in this way as compared to working directly on the underlying array.

For other fields which are built on `FieldTuples`, 1D indexing will instead index the tuple indices:

In [88]:
ft

8-element Field2Tuple{(:Q, :U), CMBLensing.BasisTuple{Tuple{Map,Map}}, Float64}:
 0.17774917746454766
 0.2511905982997429 
 0.6912012950726147 
 0.373539849474948  
 0.39985459058015005
 0.9001296871952931 
 0.41989219275007916
 0.5612169223732928 

In [89]:
ft[1]

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 0.17774917746454766
 0.2511905982997429 
 0.6912012950726147 
 0.373539849474948  

In [90]:
ft[2]

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 0.39985459058015005
 0.9001296871952931 
 0.41989219275007916
 0.5612169223732928 

In [91]:
ft[3]

BoundsError: BoundsError: attempt to access (Q = [0.17774917746454766, 0.2511905982997429, 0.6912012950726147, 0.373539849474948], U = [0.39985459058015005, 0.9001296871952931, 0.41989219275007916, 0.5612169223732928])
  at index [3]

To get the underlying data arrays, use the object's properties:

In [93]:
f.Ix

2×2 Array{Float64,2}:
 0.732042  0.182965
 0.998787  0.732902

You can always find out what properties are available by typing `f.<Tab>`. For example, if you typed `ft` then hit `<Tab>` you'd get:

In [108]:
ft |> propertynames

(:fs, :Q, :U, :Qx, :Ux)

For a `FieldTuple` like the `FlatQUMap` object, `ft`, you can get each individual Q or U field:

In [115]:
ft.Q

4-element FlatMap{2×2 map, 1′ pixels, fourier∂, Array{Float64}}:
 0.6922927799412399
 0.6961360514468293
 0.8130790577325409
 0.8044087072888599

Or `ft.Qx` which is shorthand for `ft.Q.Ix`:

In [116]:
ft.Q.Ix === ft.Qx

true

If you convert `f` to Fourier space, it would have the `Il` property to get the Fourier coefficients:

In [128]:
Fourier(f).Il

2×2 Array{Complex{Float64},2}:
  3.56432e-8+0.0im  1.09751e-8+0.0im
 -1.09983e-8+0.0im  3.81376e-9+0.0im

For convenience, you can also use the `[:...]` syntax which does the conversion automatically:

In [129]:
f[:Il]

2×2 Array{Complex{Float64},2}:
  3.56432e-8+0.0im  1.09751e-8+0.0im
 -1.09983e-8+0.0im  3.81376e-9+0.0im

This works between any bases. For example. `ft` is originally `QUMap` but we can convert to `EBFourier` and get the `El` coefficients:

In [131]:
ft[:El]

2×2 Array{Complex{Float64},2}:
 -4.04808e-8-0.0im  3.08475e-9+0.0im
 6.50065e-11+0.0im    6.633e-9+0.0im

The general rule for these two ways of accessing the underlying data is:

* **Properties** (i.e. `f.Ix`) are type-stable and get you the underlying data arrays, even recursively from `FieldTuples`. If these arrays are modified, they affect the original field.
* **Indices** (i.e. `f[:Ix]`) are not type-stable, and may not be one of the underlying data arrays (because a basis conversion may have been performed). They should be used for getting (not setting) data, and in non-performance-critical code. 